**Secrets**

The secrets below  like the Cosmos account key are retrieved from a secret scope. If you don't have defined a secret scope for a Cosmos Account you want to use when going through this sample you can find the instructions on how to create one here:
- Here you can [Create a new secret scope](./#secrets/createScope) for the current Databricks workspace
  - See how you can create an [Azure Key Vault backed secret scope](https://docs.microsoft.com/en-us/azure/databricks/security/secrets/secret-scopes#--create-an-azure-key-vault-backed-secret-scope) 
  - See how you can create a [Databricks backed secret scope](https://docs.microsoft.com/en-us/azure/databricks/security/secrets/secret-scopes#create-a-databricks-backed-secret-scope)
- And here you can find information on how to [add secrets to your Spark configuration](https://docs.microsoft.com/en-us/azure/databricks/security/secrets/secrets#read-a-secret)
If you don't want to use secrets at all you can of course also just assign the values in clear-text below - but for obvious reasons we recommend the usage of secrets.

In [ ]:
cosmosEndpoint = spark.conf.get("spark.cosmos.accountEndpoint")
cosmosMasterKey = spark.conf.get("spark.cosmos.accountKey")

**Preparation - loading data source "[NYC Taxi & Limousine Commission - green taxi trip records](https://azure.microsoft.com/en-us/services/open-datasets/catalog/nyc-taxi-limousine-commission-green-taxi-trip-records/)"**

The green taxi trip records include fields capturing pick-up and drop-off dates/times, pick-up and drop-off locations, trip distances, itemized fares, rate types, payment types, and driver-reported passenger counts. This data set has over 80 million records (>8 GB) of data and is available via a publicly accessible Azure Blob Storage Account located in the East-US Azure region.

In [ ]:
import uuid
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Azure storage access info
blob_account_name = "azureopendatastorage"
blob_container_name = "nyctlc"
blob_relative_path = "green"
blob_sas_token = r""
# Allow SPARK to read from Blob remotely
wasbs_path = 'wasbs://%s@%s.blob.core.windows.net/%s' % (blob_container_name, blob_account_name, blob_relative_path)
spark.conf.set(
  'fs.azure.sas.%s.%s.blob.core.windows.net' % (blob_container_name, blob_account_name),
  blob_sas_token)
print('Remote blob path: ' + wasbs_path)
# SPARK read parquet, note that it won't load any data yet by now
df_rawInput = spark.read.parquet(wasbs_path)

# Adding an id column with unique values
uuidUdf= udf(lambda : str(uuid.uuid4()),StringType())
df_input_withId = df_rawInput.repartition(200).withColumn("id", uuidUdf())

print('Register the DataFrame as a SQL temporary view: source')
df_input_withId.createOrReplaceTempView('source')

**Preparation - creating the Cosmos DB container to ingest the data into**

Configure the Catalog API to be used

In [ ]:
spark.conf.set("spark.sql.catalog.cosmosCatalog", "com.azure.cosmos.spark.CosmosCatalog")
spark.conf.set("spark.sql.catalog.cosmosCatalog.spark.cosmos.accountEndpoint", cosmosEndpoint)
spark.conf.set("spark.sql.catalog.cosmosCatalog.spark.cosmos.accountKey", cosmosMasterKey)

And execute the command to create the new container with a throughput of up-to 100,000 RU (Autoscale - so 10,000 - 100,000 RU based on scale) and only system properties (like /id) being indexed. We will also create a second container that will be used to store metadata for the global throughput control

In [ ]:
%sql
CREATE DATABASE IF NOT EXISTS cosmosCatalog.SampleDatabase;

CREATE TABLE IF NOT EXISTS cosmosCatalog.SampleDatabase.GreenTaxiRecords
USING cosmos.items
TBLPROPERTIES(partitionKeyPath = '/id', autoScaleMaxThroughput = '100000', indexingPolicy = 'OnlySystemProperties');

/* NOTE: It is important to enable TTL (can be off/-1 by default) on the throughput control container */
CREATE TABLE IF NOT EXISTS cosmosCatalog.SampleDatabase.ThrouhgputControl
USING cosmos.items
TBLPROPERTIES(partitionKeyPath = '/groupId', autoScaleMaxThroughput = '4000', indexingPolicy = 'AllProperties', defaultTtlInSeconds = '-1');

** Sample - ingesting the NYC Green Taxi data into Cosmos DB**

By setting the taregt throughput threshold to 0.8 (80%) we reduce throttling but still allow the ingestion to consume most of the provisioned throughput. For scenarios where ingestion shoudl only take a smaller subset of the available throughput this threshold can be reduced accordingly.

In [ ]:
writeCfg = {
  "spark.cosmos.accountEndpoint": cosmosEndpoint,
  "spark.cosmos.accountKey": cosmosMasterKey,
  "spark.cosmos.database": "SampleDatabase",
  "spark.cosmos.container": "GreenTaxiRecords",
  "spark.cosmos.write.strategy": "ItemOverwrite",
  "spark.cosmos.write.maxConcurrency": "150",
  "spark.cosmos.write.bulkEnabled": "true",
  "spark.cosmos.throughputControlEnabled": "true",
  "spark.cosmos.throughputControl.name": "NYCGreenTaxiDataIngestion",
  "spark.cosmos.throughputControl.targetThroughputThreshold": "0.8",
  "spark.cosmos.throughputControl.globalControl.database": "SampleDatabase",
  "spark.cosmos.throughputControl.globalControl.container": "ThrouhgputControl",
}

df_NYCGreenTaxi_Input = spark.sql('SELECT * FROM source')

df_NYCGreenTaxi_Input \
  .write \
  .format("cosmos.items") \
  .mode("Append") \
  .options(**writeCfg) \
  .save()

**Cleanup - deleting the Cosmos DB container and database again (to reduce cost) - skip this step if you want to keep them**

In [ ]:
%sql
DROP TABLE IF EXISTS SampleDatabase.GreenTaxiRecords;

DROP TABLE IF EXISTS SampleDatabase.ThrouhgputControl;

DROP DATABASE IF EXISTS cosmosCatalog.SampleDatabase CASCADE;